## Transfer Learning

### Install Libraries

In [ ]:
# Importing sys to check the current Python executable
import sys
print(sys.executable)

# Import necessary libraries and install if they are not already installed
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    'numpy', 'pandas', 'opencv-python', 'Pillow', 'torch', 'torchvision',
    'scikit-learn', 'jupyter'
]

# Install the required packages
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

# Now you can safely import the packages
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, precision_score, recall_score
import time # for random seed
import random
from sklearn.preprocessing import LabelEncoder



# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained VGG model
vgg = models.vgg19(pretrained=True).features.to(device).eval()

# Load pre-trained ResNet18 model
resnet18 = models.resnet18(pretrained=True)
resnet18 = torch.nn.Sequential(*(list(resnet18.children())[:-1])).to(device).eval()


print("All packages are installed and imported successfully!")


## Load the training data

### Get the id-labels mapping in the dataset

In [ ]:
# tiny-imagenet-200 dataset
data_dir = "tiny-imagenet-200"
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')



# get all of the classes whose images are in the training set
training_class_ids = os.listdir(train_dir)

# class id - class label mapping
id_class_label_map = {}
with open(os.path.join(data_dir, 'words.txt'), 'r') as f:
    for line in f.readlines():
        line = line.strip().split('\t')
        class_id = line[0]
        class_label = line[1]
        id_class_label_map[class_id] = class_label


# for each class in the tiny-imagenet-200 dataset, get the class label
training_id_class_map = {}
for class_id in training_class_ids:
    class_label = id_class_label_map[class_id]
    training_id_class_map[class_id] = class_label

# we have 200 classes in the tiny-imagenet-200 dataset with their class labels

### Get random 10 classes


In [ ]:

def get_ten_random_class_ids(training_class_ids):
    # get 10 random classes

    # seed random with time
    np.random.seed(int(time.time()))

    ten_random_classes = np.random.choice(training_class_ids, 10, replace=False)
    return ten_random_classes


print("10 random classes:")
ten_random_classes = get_ten_random_class_ids(training_class_ids)
for i in range(10):
    print(f"{i+1}. {ten_random_classes[i]} - {training_id_class_map[ten_random_classes[i]]}")

### Load images of the classes

In [ ]:
# get the images in the training set for each of the 10 random classes
# class-id to class images
training_class_images = {}
for class_id in ten_random_classes:
    class_images = os.listdir(os.path.join(train_dir, class_id, 'images'))
    training_class_images[class_id] = class_images

for class_id in ten_random_classes:
    print(f"{class_id} - {training_id_class_map[class_id]} has {len(training_class_images[class_id])} images.")


### Style-Transfer 50% of the images

In [ ]:



# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained VGG model
vgg = models.vgg19(pretrained=True).features.to(device).eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



#Function to load and transform an image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Function to convert tensor back to image
def tensor_to_image(tensor):
    if tensor.dim() == 4:  # If tensor is of shape (1, C, H, W)
        tensor = tensor.squeeze(0)
    image = tensor.clone().detach().cpu().numpy()
    image = image.transpose(1, 2, 0)
    image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    image = image.clip(0, 1)
    return Image.fromarray((image * 255).astype('uint8'))

    

# Function to compute Gram matrix
def gram_matrix(input):
    batch_size, feature_maps, h, w = input.size()
    features = input.view(batch_size * feature_maps, h * w)
    G = torch.mm(features, features.t())
    return G.div(batch_size * feature_maps * h * w)

# Define content and style layers
content_layers = ['0']
style_layers = ['0', '5', '10', '19', '28']

# Function to extract features
def get_features(image, model, layers):
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[name] = x
    return features


def style_transfer(content_img, style_img, model, content_weight=1, style_weight=1e6, num_steps=300):
    content_features = get_features(content_img, model, content_layers)
    style_features = get_features(style_img, model, style_layers)
    
    target = content_img.clone().requires_grad_(True).to(device)
    optimizer = torch.optim.Adam([target], lr=0.003)
    
    for step in range(num_steps):
        target_features = get_features(target, model, content_layers + style_layers)
        
        content_loss = F.mse_loss(target_features['0'], content_features['0'])
        style_loss = 0
        for layer in style_layers:
            style_loss += F.mse_loss(gram_matrix(target_features[layer]), gram_matrix(style_features[layer]))
        
        total_loss = content_weight * content_loss + style_weight * style_loss
        
        optimizer.zero_grad()
        total_loss.backward(retain_graph=True)  # Retain the graph
        optimizer.step()
        
        if step % 50 == 0:
            print(f"Step {step}/{num_steps}, Content Loss: {content_loss.item()}, Style Loss: {style_loss.item()}")
    
    return target


output_dir = 'styled_images'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)


# Process images for style transfer
for class_id in ten_random_classes:
    class_dir = os.path.join(train_dir, class_id, 'images')
    image_names = training_class_images[class_id]
    
    # Randomly select 50% of the images for style transfer
    selected_images = random.sample(image_names, len(image_names) // 2)
    
    for image_name in image_names:
        img_path = os.path.join(class_dir, image_name)
        content_image = load_image(img_path)
        
        if image_name in selected_images:
            style_img_path = os.path.join(class_dir, random.choice(image_names))
            style_image = load_image(style_img_path)
            styled_image = style_transfer(content_image, style_image, vgg, num_steps=100)
            output_image = tensor_to_image(styled_image)
            output_image.save(os.path.join(output_dir, f"{class_id}_styled_{image_name}"))
        else:
            output_image = tensor_to_image(content_image.squeeze(0))
            output_image.save(os.path.join(output_dir, f"{class_id}_original_{image_name}"))
        

## Extract Feature and Prepare Dataset

In [ ]:


# Function to extract features using ResNet18
def extract_features(images, model):
    features = []
    for image in images:
        with torch.no_grad():
            feature = model(image.unsqueeze(0))  # Add batch dimension
            feature = feature.view(feature.size(0), -1)  # Flatten features
            features.append(feature.cpu().numpy())
    return np.array(features)



# Extract features for all images and prepare dataset
X = []
y = []

for class_id in ten_random_classes:
    for image_name in training_class_images[class_id]:
        if f"{class_id}_styled_{image_name}" in os.listdir(output_dir):
            img_path = os.path.join(output_dir, f"{class_id}_styled_{image_name}")
        else:
            img_path = os.path.join(output_dir, f"{class_id}_original_{image_name}")
        
        image = load_image(img_path)
        features = extract_features([image], resnet18)
        X.append(features[0])
        y.append(class_id)

X = np.array(X)
y = np.array(y)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


## 

In [ ]:
# Train logistic regression classifier
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X, y_encoded)

# Evaluate the classifier
y_pred = classifier.predict(X)
accuracy = accuracy_score(y_encoded, y_pred)
precision = precision_score(y_encoded, y_pred, average='weighted')
recall = recall_score(y_encoded, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
